In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(11, 4),
    titlesize=18,
    titleweight='bold',
)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
)
%config InlineBackend.figure_format = 'retina'

In [ ]:
# load Apple data
apple = pd.read_csv('../input/stock-time-series-20050101-to-20171231/AAPL_2006-01-01_to_2018-01-01.csv', index_col="Date", parse_dates=["Date"])

In [ ]:
apple.head()

In [ ]:
apple.tail()

Context: 

Date - in format: yy-mm-dd

Open - price of the stock at market open (this is NYSE data so all in USD)

High - Highest price reached in the day

Low Close - Lowest price reached in the day

Volume - Number of shares traded

Name - the stock's ticker name

In [ ]:
apple.shape

In [ ]:
apple.info()

In [ ]:
apple.plot(subplots=True, figsize=(10, 15))

In [ ]:
# .asfreq( ) does change the size of series
# Converts time series to specified frequency.
# Returns the original data conformed to a new index with the specified frequency
apple["Close"].asfreq('M').interpolate().plot(legend=True)
shifted = apple["Close"].asfreq('M').interpolate().shift(10).plot(legend=True)
plt.legend(['Close','Close_lagged'])
plt.title('Closing price of Apple over time (Monthly frequency) - raw and lagged')
plt.show()

**Resampling time series:**

Undersampling - Time series is resampled from high frequency to low frequency (here weekly to monthly frequency). It needs aggregation.

Oversampling - Time series is resampled from low frequency to high frequency (here daily to hourly frequency). It needs filling or interpolating missing data



In [ ]:
# Undersampling (decreases the size of series)
print(apple.shape[0])
print(apple.resample('M').mean().shape[0])
(apple.resample('M').mean())["Close"].plot()
plt.title("'Closing price of Apple over time (Monthly frequency) - undersampled'")

In [ ]:
# Oversampling (increases the size of series)
apple.resample("H").fillna(method="ffill")

#apple.resample("H").fillna(method="bfill")

# apple.resample("H").interpolate( )

**Window functions:**

Rolling - Same size and sliding

Expanding - Contains all prior values

**Moving average plots to discover the trend in a series:**

In [ ]:
# rolling does not change the size of series
print(apple.shape[0])
print(apple["Close"].rolling(window = 30).mean().shape)
apple["Close"].plot()
apple["Close"].rolling(window = 30).mean().plot()
plt.legend(["Close", "Rolling Mean"])

In [ ]:
# Moving average plot of apple["Close"]
apple["Close"].plot()
#apple["Close"].rolling(window = "180D").mean().plot()
moving_average = apple["Close"].rolling(
    window=360,       # 180-day window
    center=True,      # puts the average at the center of the window
    min_periods=180,  # choose about half the window size
).mean()              # compute the mean (could also do median, std, min, max, ...)
moving_average.plot()
plt.legend(["Close", "Moving average"])

In [ ]:
# Moving average plot of apple["Volume"]
apple["Volume"].plot()
#apple["Close"].rolling(window = "180D").mean().plot()
moving_average = apple["Volume"].rolling(
    window=360,       # 180-day window
    center=True,      # puts the average at the center of the window
    min_periods=180,  # choose about half the window size
).mean()              # compute the mean (could also do median, std, min, max, ...)
moving_average.plot()
plt.legend(["Volume", "Moving average"])

**Autocorrelation and Partial Autocorrelation:**

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


The most commonly used measure of serial dependence is known as autocorrelation, which is the correlation a time series has with one of its lags.

In [ ]:
# Autocorrelation of Closing price of Apple
plot_acf(apple["Close"],lags=20,title="Autocorrelation chart: Apple (Close price)")
plot_acf(apple["Volume"],lags=20,title="Autocorrelation chart: Apple (Volume)")
plt.show()

All lags are greater than the confidence interval, so they are statistically significant.

 The **partial autocorrelation** tells you the correlation of a lag **accounting for all of the previous lags**, i.e., the amount of "new" correlation the lag contributes.


In [ ]:
# Partial Autocorrelation of Closing price of Apple
_ = plot_pacf(apple["Close"],lags=20, title="Partial autocorrelation of Apple (Close price) with 95% confidence intervals of no correlation.")
plt.show()

Partial autocorrelation after first lag is very low.

Note taht autocorrelation and partial autocorrelation are measures of linear dependence. Because time series may have non-linear dependences, we need to look at a lag plot (or a more general measure of dependence, like mutual information) when choosing lag features.



In [ ]:
# the 2 follwing functions taken from https://www.kaggle.com/ryanholbrook/time-series-as-features

def lagplot(x, y=None, lag=1, standardize=False, ax=None, **kwargs):
    from matplotlib.offsetbox import AnchoredText
    x_ = x.shift(lag)
    if standardize:
        x_ = (x_ - x_.mean()) / x_.std()
    if y is not None:
        y_ = (y - y.mean()) / y.std() if standardize else y
    else:
        y_ = x
    corr = y_.corr(x_)
    if ax is None:
        fig, ax = plt.subplots()
    scatter_kws = dict(
        alpha=0.75,
        s=3,
    )
    line_kws = dict(color='C3', )
    ax = sns.regplot(x=x_,
                     y=y_,
                     scatter_kws=scatter_kws,
                     line_kws=line_kws,
                     lowess=True,
                     ax=ax,
                     **kwargs)
    at = AnchoredText(
        f"{corr:.2f}",
        prop=dict(size="large"),
        frameon=True,
        loc="upper left",
    )
    at.patch.set_boxstyle("square, pad=0.0")
    ax.add_artist(at)
    ax.set(title=f"Lag {lag}", xlabel=x_.name, ylabel=y_.name)
    return ax



def plot_lags(x, y=None, lags=6, nrows=1, lagplot_kwargs={}, **kwargs):
    import math
    kwargs.setdefault('nrows', nrows)
    kwargs.setdefault('ncols', math.ceil(lags / nrows))
    kwargs.setdefault('figsize', (kwargs['ncols'] * 2, nrows * 2 + 0.5))
    fig, axs = plt.subplots(sharex=True, sharey=True, squeeze=False, **kwargs)
    for ax, k in zip(fig.get_axes(), range(kwargs['nrows'] * kwargs['ncols'])):
        if k + 1 <= lags:
            ax = lagplot(x, y, lag=k + 1, ax=ax, **lagplot_kwargs)
            ax.set_title(f"Lag {k + 1}", fontdict=dict(fontsize=14))
            ax.set(xlabel="", ylabel="")
        else:
            ax.axis('off')
    plt.setp(axs[-1, :], xlabel=x.name)
    plt.setp(axs[:, 0], ylabel=y.name if y is not None else x.name)
    fig.tight_layout(w_pad=0.1, h_pad=0.1)
    return fig


In [ ]:
# A lag plot of a time series shows its values plotted against its lags
_ = plot_lags(apple["Close"], lags=20, nrows=3)

The lag plots indicate that the relationship of apple["Close"] to its lags is linear, while the partial autocorrelations suggest the dependence can be captured using lags 1.

In [ ]:
# Partial Autocorrelation of Volume of Apple
_ = plot_pacf(apple["Volume"],lags=20, title="Partial autocorrelation of Apple (Volume) with 95% confidence intervals of no correlation.")
plt.show()

Partial autocorrelation after 11th lag is very low.

Note taht partial autocorrelation are measures of linear dependence. Because time series may have non-linear dependences, we need to look at a lag plot (or a more general measure of dependence, like mutual information) when choosing lag features.

In [ ]:
# A lag plot of a time series shows its values plotted against its lags
_ = plot_lags(apple["Volume"], lags=20, nrows=3)

**Choosing lags as features for ML models:**

In [ ]:
# Use lags as features for ML models

def make_lags(ts, lags):
    return pd.concat(
        {
            f'y_lag_{i}': ts.shift(i)
            for i in range(1, lags + 1)
        },
        axis=1)



In [ ]:
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse


**1. Linear Regression and Lag features for apple["Close"]:**

In [ ]:
# Create X and y for ML models to predict apple["Close"] 
X = make_lags(apple["Close"], lags=1)
X = X.fillna(0.0)
y = apple["Close"].copy()

In [ ]:
# data splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Fit and predict
model = LinearRegression()  # `fit_intercept=True` since we didn't use DeterministicProcess
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_train), index=y_train.index)
y_fore = pd.Series(model.predict(X_test), index=y_test.index)
print(f'Model train accuracy: {model.score(X_train, y_train)*100:.3f}%')
print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')
print(f'Model train MAE: {mae(y_pred,y_train):.3f}')
print(f'Model train RMSE: {mse(y_pred,y_train, squared=False):.3f}')
print(f'Model test MAE: {mae(y_fore,y_test):.3f}')
print(f'Model test RMSE: {mse(y_fore,y_test, squared=False):.3f}')

In [ ]:
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
)

In [ ]:
y_train.plot(**plot_params)
y_test.plot(**plot_params)
y_pred.plot( )
y_fore.plot( )

**1. Linear Regression and Lag features for apple["Volume"]:**

In [ ]:
# Create X and y for ML models to predict apple["Volume"] 
X1 = make_lags(apple["Volume"], lags=11)
X1 = X1.fillna(0.0)

X2 = pd.DataFrame(apple["Volume"].shift(13)).rename(columns={"Volume":"y_lag_13"})
X2 = X2.fillna(0.0)

X3 = pd.DataFrame(apple["Volume"].shift(15)).rename(columns={"Volume":"y_lag_15"})
X3 = X3.fillna(0.0)

X4 = pd.DataFrame(apple["Volume"].shift(18)).rename(columns={"Volume":"y_lag_18"})
X4 = X4.fillna(0.0)

X5 = pd.DataFrame(apple["Volume"].shift(19)).rename(columns={"Volume":"y_lag_19"})
X5 = X5.fillna(0.0)

X = pd.concat([X1, X2, X3, X4, X5], axis=1)

y = apple["Volume"].copy()

In [ ]:
# data splits
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False, random_state= 101)

# Fit and predict
model = LinearRegression()  # `fit_intercept=True` since we didn't use DeterministicProcess

model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_train), index=y_train.index)
y_fore = pd.Series(model.predict(X_test), index=y_test.index)
print(f'Model train accuracy: {model.score(X_train, y_train)*100:.3f}%')
print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')
print(f'Model train MAE: {mae(y_pred,y_train):.3f}')
print(f'Model train RMSE: {mse(y_pred,y_train, squared=False):.3f}')
print(f'Model test MAE: {mae(y_fore,y_test):.3f}')
print(f'Model test RMSE: {mse(y_fore,y_test, squared=False):.3f}')

In [ ]:
y_train.plot(**plot_params)
y_test.plot(**plot_params)
y_pred.plot( )
y_fore.plot( )

**2. Hybrid model (Linear Regression + XGBoost) for apple["Volume"]:** 

model_1 = Linear Regression and DeterministicProcess as features to find trend 

model_2 = XGBoost and Lag features 

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess


In [ ]:
# 1. Train and predict with first model (LinearRegression) 
# a simple (usually linear) learning algorithm
# model_1.fit(X_train_1, y_train)
# y_pred_1 = model_1.predict(X_train)

y = apple["Volume"].copy()

dp = DeterministicProcess(
    index=y.index,  # dates from the training data
    constant=True,  # the intercept
    order=1,        # linear trend
    drop=True,      # drop terms to avoid collinearity
)
X = dp.in_sample()  # features for the training data for the first model

# It will be easier for us later if we
# split the date index instead of the dataframe directly.
idx_train, idx_test = train_test_split(y.index, test_size=0.2, shuffle=False)
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]

# Fit trend model
model = LinearRegression(fit_intercept=False) # `fit_intercept=False` since we did use DeterministicProcess with constant=True 
model.fit(X_train, y_train)

# Make predictions
y_fit = pd.Series(
    model.predict(X_train),
    index=y_train.index,
    )
y_pred = pd.Series(
    model.predict(X_test),
    index=y_test.index,
    )

# Plot
axs = y_train.plot(color='0.25', subplots=True, sharex=True)
axs = y_test.plot(color='0.25', subplots=True, sharex=True, ax=axs)
axs = y_fit.plot(color='C0', subplots=True, sharex=True, ax=axs)
axs = y_pred.plot(color='C3', subplots=True, sharex=True, ax=axs)
for ax in axs: ax.legend([])
_ = plt.suptitle("Volume Trend")

In [ ]:
# 2. Train and predict with second model on residuals
# model_2.fit(X_train_2, y_train - y_pred_1)
# y_pred_2 = model_2.predict(X_train_2)


#X = make_lags(apple["Volume"], lags=11)
#X = X.fillna(0.0)

# Create X for the second ML model 
X1 = make_lags(apple["Volume"], lags=11)
X1 = X1.fillna(0.0)

X2 = pd.DataFrame(apple["Volume"].shift(13)).rename(columns={"Volume":"y_lag_13"})
X2 = X2.fillna(0.0)

X3 = pd.DataFrame(apple["Volume"].shift(15)).rename(columns={"Volume":"y_lag_15"})
X3 = X3.fillna(0.0)

X4 = pd.DataFrame(apple["Volume"].shift(18)).rename(columns={"Volume":"y_lag_18"})
X4 = X4.fillna(0.0)

X5 = pd.DataFrame(apple["Volume"].shift(19)).rename(columns={"Volume":"y_lag_19"})
X5 = X5.fillna(0.0)

X = pd.concat([X1, X2, X3, X4, X5], axis=1)


# Create splits
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
# y_train, y_test = y.loc[idx_train], y.loc[idx_test]

In [ ]:
# 2. Train and predict with second model (a complex, non-linear learner like GBDTs or a deep neural net) 
# on residuals
# model_2.fit(X_train_2, y_train - y_pred_1)
# y_pred_2 = model_2.predict(X_train_2)

# Create residuals (the collection of detrended series) from the training set
y_resid = y_train - y_fit

# Train XGBoost on the residuals
xgb = XGBRegressor()
xgb.fit(X_train, y_resid)

# Add the predicted residuals onto the predicted trends
y_fit_boosted = xgb.predict(X_train) + y_fit
y_pred_boosted = xgb.predict(X_test) + y_pred

In [ ]:
print(f'Model train MAE: {mae(y_fit_boosted,y_train):.3f}')
print(f'Model test MAE: {mae(y_pred_boosted,y_test):.3f}')

print(f'Model train RMSE: {mse(y_fit_boosted,y_train, squared=False):.3f}')
print(f'Model test RMSE: {mse(y_pred_boosted,y_test, squared=False):.3f}')

In [ ]:
axs = y_train.plot(color='0.25', figsize=(11, 5), subplots=True, sharex=True, title=['Volume'])
axs = y_test.plot(color='0.25', subplots=True, sharex=True, ax=axs)
axs = y_fit_boosted.plot(color='C0', subplots=True, sharex=True, ax=axs)
axs = y_pred_boosted.plot(color='C3', subplots=True, sharex=True, ax=axs)
for ax in axs: ax.legend([])

**Time series decomposition to:** 

Trend - Consistent upwards or downwards slope of a time series

Seasonality - Clear periodic pattern of a time series

Noise - Outliers or missing values

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

# Period of the series. Must be used if x is not a pandas object or if the index of x does not have a frequency. Overrides default periodicity of x if x is a pandas object with a timeseries index.
decomposed_oracle_close = seasonal_decompose(apple["Close"], period=180) 
decomposed_oracle_close.plot( )
plt.show()

**Seasonality:** (text is taken from https://www.kaggle.com/ryanholbrook/seasonality)

A time series exhibits seasonality whenever there is a regular, periodic change in the mean of the series.

Two kinds of features that model seasonality:

Indicators: for a short season with few observations, like a weekly season of daily observations.
Seasonal indicators are binary features, and they are what you get if you treat a seasonal period as a categorical feature and apply one-hot encoding.

Fourier features: for a long season with many observations, like an annual season of daily observations. Instead of creating a feature for each date, Fourier features capture the overall shape of the seasonal curve with just a few features.

Fourier features are pairs of sine and cosine curves, one pair for each potential frequency in the season starting with the longest.

How many Fourier pairs should we include in our feature set? We can answer this question with the periodogram. The periodogram tells you the strength of the frequencies in a time series. The value on the y-axis is (a ** 2 + b ** 2) / 2, where a and b are the coefficients of the sine and cosine at that frequency.



In [ ]:
# The follwing function is taken from https://www.kaggle.com/ryanholbrook/seasonality

def plot_periodogram(ts, detrend='linear', ax=None):
    from scipy.signal import periodogram
    fs = pd.Timedelta("1Y") / pd.Timedelta("1D")
    freqencies, spectrum = periodogram(
        ts,
        fs=fs,
        detrend=detrend,
        window="boxcar",
        scaling='spectrum',
    )
    if ax is None:
        _, ax = plt.subplots()
    ax.step(freqencies, spectrum, color="purple")
    ax.set_xscale("log")
    ax.set_xticks([1, 2, 4, 6, 12, 26, 52, 104])
    ax.set_xticklabels(
        [
            "Annual (1)",
            "Semiannual (2)",
            "Quarterly (4)",
            "Bimonthly (6)",
            "Monthly (12)",
            "Biweekly (26)",
            "Weekly (52)",
            "Semiweekly (104)",
        ],
        rotation=30,
    )
    ax.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
    ax.set_ylabel("Variance")
    ax.set_title("Periodogram")
    return ax

In [ ]:
plot_periodogram(apple.Close)
plot_periodogram(apple.Volume)

**Seasonal plots:**

In [ ]:
# The follwing function is taken from https://www.kaggle.com/ryanholbrook/seasonality

def seasonal_plot(X, y, period, freq, ax=None):
    if ax is None:
        _, ax = plt.subplots()
    palette = sns.color_palette("husl", n_colors=X[period].nunique(),)
    ax = sns.lineplot(
        x=freq,
        y=y,
        hue=period,
        data=X,
        ci=False,
        ax=ax,
        palette=palette,
        legend=False,
    )
    ax.set_title(f"Seasonal Plot ({period}/{freq})")
    for line, name in zip(ax.lines, X[period].unique()):
        y_ = line.get_ydata()[-1]
        ax.annotate(
            name,
            xy=(1, y_),
            xytext=(6, 0),
            color=line.get_color(),
            xycoords=ax.get_yaxis_transform(),
            textcoords="offset points",
            size=14,
            va="center",
        )
    return ax

In [ ]:
# Seasonal plots for apple.Close

X = apple.copy()
# days within a week
X["dayofweek"] = X.index.dayofweek  # the x-axis (freq)
X["week"] = X.index.week  # the seasonal period (period)

# days within a month
X["dayofmonth"] = X.index.day
X["month"] = X.index.month

# days within a year
X["dayofyear"] = X.index.dayofyear
X["year"] = X.index.year

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(11, 6))
seasonal_plot(X, y="Close", period="week", freq="dayofweek", ax=ax0)
seasonal_plot(X, y="Close", period="month", freq="dayofmonth", ax=ax1);
seasonal_plot(X, y="Close", period="year", freq="dayofyear", ax=ax2);


In [ ]:
# Seasonal plots for apple.Volume

X = apple.copy()
# days within a week
X["dayofweek"] = X.index.dayofweek  # the x-axis (freq)
X["week"] = X.index.week  # the seasonal period (period)

# days within a month
X["dayofmonth"] = X.index.day
X["month"] = X.index.month

# days within a year
X["dayofyear"] = X.index.dayofyear
X["year"] = X.index.year

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, figsize=(11, 6))
seasonal_plot(X, y="Volume", period="week", freq="dayofweek", ax=ax0)
seasonal_plot(X, y="Volume", period="month", freq="dayofmonth", ax=ax1);
seasonal_plot(X, y="Volume", period="year", freq="dayofyear", ax=ax2);


**Linear Regression and concatenation of lag features and deterministic process as features to predict apple["Close"]**

In [ ]:
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

fourier = CalendarFourier(freq="A", order=3)  # 3 sin/cos pairs for "A"nnual seasonality



In [ ]:
apple.index

In [ ]:
# we change timestamp to period in order index to have a frequency
#apple.index = apple.index.to_period("D")
#apple.index

In [ ]:
#apple.index = apple.index.to_timestamp("D")
#apple.index

In [ ]:
dp = DeterministicProcess(
    index=apple.index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    #seasonal=True,              # seasonality (indicators). 
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

# Note If period in DeterministicProcess not provided, 
# for the period of the seasonal (or fourier) components, freq is read from index if available.
# freq of index is "D" here (apple.index = apple.index.to_period("D")). 


X_t_s = dp.in_sample()  # create features for dates in apple.index

In [ ]:
X_t_s.head()

In [ ]:
X_c = make_lags(apple["Close"], lags=1)
X_c = X_c.fillna(0.0)

In [ ]:
X = pd.concat([X_t_s, X_c], axis=1)

In [ ]:
# Create target series and data splits
y = apple["Close"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Fit and predict
model = LinearRegression(fit_intercept=False)  # `fit_intercept=False` since we uses DeterministicProcess with constant=True
model.fit(X_train, y_train)
y_pred = pd.Series(model.predict(X_train), index=y_train.index)
y_fore = pd.Series(model.predict(X_test), index=y_test.index)
print(f'Model train accuracy: {model.score(X_train, y_train)*100:.3f}%')
print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')

print(f'Model train MAE: {mae(y_pred,y_train):.3f}')
print(f'Model train RMSE: {mse(y_pred,y_train, squared=False):.3f}')
print(f'Model test MAE: {mae(y_fore,y_test):.3f}')
print(f'Model test RMSE: {mse(y_fore,y_test, squared=False):.3f}')

In [ ]:
y_train.plot(**plot_params)
y_test.plot(**plot_params)
y_pred.plot( )
y_fore.plot( )

**Linear Regression and DeterministicProcess as features to predict trend for apple["Close"]**

In [ ]:
apple.index

In [ ]:
apple.index = apple.index.to_period("D")
apple.index

In [ ]:
y = apple["Close"].copy()

fourier = CalendarFourier(freq="A", order=3)  # 3 sin/cos pairs for "A"nnual seasonality

dp = DeterministicProcess(
    index=apple.index,
    constant=True,               # dummy feature for bias (y-intercept)
    order=1,                     # trend (order 1 means linear)
    #seasonal=True,              # seasonality (indicators). 
    additional_terms=[fourier],  # annual seasonality (fourier)
    drop=True,                   # drop terms to avoid collinearity
)

X = dp.in_sample()  # create features for dates in apple.index

model = LinearRegression(fit_intercept=False) # `fit_intercept=False` since we uses DeterministicProcess with constant=True
model.fit(X, y)
y_pred = pd.Series(model.predict(X), index=y.index)

X_fore = dp.out_of_sample(steps=90)
y_fore = pd.Series(model.predict(X_fore), index=X_fore.index)

ax = y.plot(color='0.25', style='.', title="Apple Close - Trend-Seasonal Forecast")
ax = y_pred.plot(ax=ax, label="Trend-Seasonal")
ax = y_fore.plot(ax=ax, label="Trend-Seasonal Forecast", color='C3')
_ = ax.legend()


**2. Hybrid model (Linear Regression + XGBoost) for apple[["Close", "Volume"]]:**

In [ ]:
# 1. Train and predict with first model (LinearRegression) 
# a simple (usually linear) learning algorithm
# model_1.fit(X_train_1, y_train)
# y_pred_1 = model_1.predict(X_train)

y = apple[["Close", "Volume"]].copy()

# Create trend features
dp = DeterministicProcess(
    index=y.index,  # dates from the training data
    constant=True,  # the intercept
    order=1,        # linear trend
    drop=True,      # drop terms to avoid collinearity
)
X = dp.in_sample()  # features for the training data

# It will be easier for us later if we
# split the date index instead of the dataframe directly.
idx_train, idx_test = train_test_split(y.index, test_size=0.2, shuffle=False)
X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]

# Fit trend model
model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)

# Make predictions
y_fit = pd.DataFrame(
    model.predict(X_train),
    index=y_train.index,
    columns=y_train.columns,
)
y_pred = pd.DataFrame(
    model.predict(X_test),
    index=y_test.index,
    columns=y_test.columns,
)

# Plot
axs = y_train.plot(color='0.25', subplots=True, sharex=True)
axs = y_test.plot(color='0.25', subplots=True, sharex=True, ax=axs)
axs = y_fit.plot(color='C0', subplots=True, sharex=True, ax=axs)
axs = y_pred.plot(color='C3', subplots=True, sharex=True, ax=axs)
for ax in axs: ax.legend([])
_ = plt.suptitle("Trends")

In [ ]:
# The `stack` method converts column labels to row labels, pivoting from wide format to long
df = apple[["Close", "Volume"]].copy()
df = pd.concat({'Value': df}, names=[None, 'Variables'], axis=1)
X = df.stack()  # pivot dataset wide to long
display(X.head())
y = X.pop('Value')  # grab target series
display(X.head())
display(y.head())

In [ ]:
# Turn row labels into categorical feature columns with a label encoding
X = X.reset_index('Variables')
display(X.head())

In [ ]:
# Label encoding for 'Variables' feature
for colname in X.select_dtypes(["object", "category"]):
    X[colname], _ = X[colname].factorize()

In [ ]:
display(X.head())

In [ ]:
# Label encoding for annual seasonality
#X["Month"] = X.index.month  # values are 1, 2, ..., 12
#X["Year"] = X.index.year

# Create splits
#X_train, X_test = pd.concat([X.loc[idx_train, :], X_c.loc[idx_train, :]], axis=1), pd.concat([X.loc[idx_test, :], X_c.loc[idx_test, :]], axis=1)

X_train, X_test = X.loc[idx_train, :], X.loc[idx_test, :]
y_train, y_test = y.loc[idx_train], y.loc[idx_test]
display(X_train.head())
display(y_train.head())

In [ ]:
# y_fit.stack().head()

In [ ]:
# y_resid = y_train - y_fit.stack()
# y_resid.head()

In [ ]:
from xgboost import XGBRegressor

# Pivot wide to long (stack) and convert DataFrame to Series (squeeze)
y_fit = y_fit.stack().squeeze()  # trend from training set
y_pred = y_pred.stack().squeeze()  # trend from test set

# Create residuals (the collection of detrended series) from the training set
y_resid = y_train - y_fit

# Train XGBoost on the residuals
xgb = XGBRegressor()
xgb.fit(X_train, y_resid)

# Add the predicted residuals onto the predicted trends
y_fit_boosted = xgb.predict(X_train) + y_fit
y_pred_boosted = xgb.predict(X_test) + y_pred

In [ ]:
axs = y_train.unstack(['Variables']).plot(
    color='0.25', figsize=(11, 5), subplots=True, sharex=True,
    title=['Close', 'Volume'],
)
axs = y_test.unstack(['Variables']).plot(color='0.25', subplots=True, sharex=True, ax=axs)

axs = y_fit_boosted.unstack( ).plot(color='C0', subplots=True, sharex=True, ax=axs)

axs = y_pred_boosted.unstack( ).plot(color='C3', subplots=True, sharex=True, ax=axs)
for ax in axs: ax.legend([])